In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import SAGEConv
import copy
import torch
from torch.utils.data import DataLoader, Dataset
import csv
import json
import pandas as pd
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
torch.manual_seed(0)

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Building graph from the edge information

from tqdm import tqdm
import networkx as nx

node_list_exp = []

G = nx.Graph()
with open('/content/drive/MyDrive/GSPPI_combined_train/Edge_combined.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        node_list_exp.append(row[0])
        node_list_exp.append(row[1])

print('len',len(node_list_exp))
set_list = set(node_list_exp)
list_set = list(set_list)
print(len(list_set))


17647it [00:00, 973662.94it/s]

len 35294
7576


In [ ]:
excel_file = "/content/drive/MyDrive/GSPPI_combined_train/MP_Hippie_combined.xlsx"
df_prot_id = pd.read_excel(excel_file)

# Create the dictionary from the DataFrame
key_value_dict = {}
for index, row in df_prot_id.iterrows():
    key = row['Key']
    value = row['Id']
    key_value_dict[key] = value

# Display the recreated dictionary
print(key_value_dict)

{'A0A087X179': 0, 'A0A087X1G2': 1, 'A0A0G2JMD5': 2, 'A0A0J9YX06': 3, 'A0A0U1RQE8': 4, 'A0A1B0GUJ8': 5, 'A0A1B0GVH7': 6, 'A0A1B0GX68': 7, 'A0A1W2PR19': 8, 'A0AUZ9': 9, 'A0AVI2': 10, 'A0AVT1': 11, 'A0FGR9': 12, 'A0JLT2': 13, 'A0JNW5': 14, 'A0PJY2': 15, 'A1A5B4': 16, 'A1A5C7': 17, 'A1KZ92': 18, 'A2A3K4': 19, 'A2A3N6': 20, 'A2PYH4': 21, 'A2RRH5': 22, 'A2RRP1': 23, 'A2RU49': 24, 'A2RUH7': 25, 'A2RUT3': 26, 'A2VCK2': 27, 'A3KMH1': 28, 'A3KN83': 29, 'A4D0V7': 30, 'A4D1E9': 31, 'A4FU69': 32, 'A5D8V6': 33, 'A5D8W1': 34, 'A5PLK6': 35, 'A5YKK6': 36, 'A6H8M9': 37, 'A6NDA9': 38, 'A6NEC2': 39, 'A6NER0': 40, 'A6NFC5': 41, 'A6NFQ2': 42, 'A6NFU0': 43, 'A6NFY4': 44, 'A6NH21': 45, 'A6NHL2': 46, 'A6NHM9': 47, 'A6NHR9': 48, 'A6NI28': 49, 'A6NIE9': 50, 'A6NIH7': 51, 'A6NIM6': 52, 'A6NKB5': 53, 'A6NKD9': 54, 'A6NKT7': 55, 'A6NLU0': 56, 'A6NM28': 57, 'A6NMZ7': 58, 'A6NNS2': 59, 'A6NNY8': 60, 'A6PVS8': 61, 'A7MCY6': 62, 'A8K0Z3': 63, 'A8K2U0': 64, 'A8K7I4': 65, 'A8K8P3': 66, 'A8MT19': 67, 'A8MT65': 68, 'A8MT69

In [ ]:
# Building graph from the edge information

from tqdm import tqdm
import networkx as nx

node_list_1 = []
node_list_2 = []
G = nx.Graph()
with open('/content/drive/MyDrive/GSPPI_combined_train/Edge_combined.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        G.add_node(key_value_dict[row[0]]) #protein1 in first column
        G.add_node(key_value_dict[row[1]]) #protein2 in second column
        node_list_1.append(key_value_dict[row[0]])
        node_list_2.append(key_value_dict[row[1]])
        if G.has_edge(key_value_dict[row[0]], key_value_dict[row[1]]):
            # edge already exists, increase weight by one
            G[key_value_dict[row[0]]][key_value_dict[row[1]]]['weight'] += 1
        else:
            # add new edge with weight 1
            G.add_edge(key_value_dict[row[0]], key_value_dict[row[1]], weight = 1)

G_nodes = G.number_of_nodes()
G_edges = G.number_of_edges()
print("Nodes = ", G_nodes, " Edges = ",G_edges )

17647it [00:00, 224539.22it/s]

Nodes =  7576  Edges =  17647


In [ ]:
edges = [(source, target) for source, target in G.edges]

# Convert the edge index to a PyTorch tensor with shape [2, num_edges]
edge_index = torch.tensor(edges, dtype=torch.long).t()
print(edge_index)
print(edge_index.shape)

tensor([[3696, 3696, 3696,  ..., 5270, 5270, 5270],
        [1136, 1409, 1790,  ...,  818, 4743, 7166]])
torch.Size([2, 17647])


In [ ]:
# combine all nodes in a list
node_list = node_list_1 + node_list_2
print(node_list)

# remove duplicate items from the list
node_list = list(dict.fromkeys(node_list))
print(node_list)

# build adjacency matrix
adj_G = nx.to_numpy_array(G, nodelist = node_list)
#adj_G.shape
#print(adj_G)


[3696, 3035, 149, 5977, 6470, 7069, 6996, 1821, 1691, 1610, 2223, 2368, 2916, 3470, 3681, 3948, 3428, 3025, 406, 460, 377, 3459, 6268, 1969, 1725, 1242, 1981, 1691, 3259, 2163, 2021, 1778, 2329, 1409, 2330, 3696, 1285, 2398, 149, 3568, 756, 7287, 7191, 2339, 2368, 3090, 5412, 1041, 2403, 3456, 1087, 2561, 5598, 5585, 7343, 2206, 1576, 2527, 2528, 1603, 5329, 2261, 1821, 1618, 2021, 1242, 2865, 1901, 1242, 1977, 1242, 2022, 3241, 2469, 1242, 1964, 3290, 2815, 4429, 5330, 6531, 1656, 1190, 3478, 1249, 1324, 1780, 1392, 1725, 5450, 2021, 3618, 1625, 3547, 2386, 3933, 5471, 1570, 2991, 3932, 3337, 2387, 3075, 3309, 1681, 6623, 1682, 5485, 3004, 3700, 3971, 3199, 5335, 5367, 3532, 2386, 1570, 3075, 3971, 2386, 1570, 1681, 5367, 5947, 3832, 3841, 233, 3949, 7207, 6653, 558, 5450, 1724, 3949, 909, 7207, 6653, 3532, 3933, 3075, 6454, 1691, 2367, 995, 1478, 3260, 5569, 2368, 1603, 2022, 352, 3470, 3428, 2398, 468, 2966, 149, 2456, 3011, 7489, 6120, 6395, 5285, 5262, 3966, 3525, 790, 2667, 6045,

In [ ]:
tensor_adj = torch.from_numpy(adj_G )
tensor_adj = tensor_adj.to(torch.long)
print(tensor_adj)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [ ]:
from tqdm import tqdm
import networkx as nx

In [ ]:
pos_train_list = []
with open('/content/drive/MyDrive/GSPPI_combined_train/combined_train_edge_pos_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        pos_train_list.append(tuple)

train_pos_edge = torch.tensor(pos_train_list, dtype=torch.long)
print(train_pos_edge)

14117it [00:00, 440194.40it/s]

tensor([[3696, 1136],
        [3696, 1409],
        [3696, 1790],
        ...,
        [1848, 2859],
        [4826, 5266],
        [7348, 5270]])


In [ ]:
neg_train_list = []
with open('/content/drive/MyDrive/GSPPI_combined_train/combined_train_edge_neg_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        neg_train_list.append(tuple)

train_neg_edge = torch.tensor(neg_train_list, dtype=torch.long)
print(train_neg_edge)

14117it [00:00, 404176.10it/s]

tensor([[2700,  961],
        [ 975, 2572],
        [4068, 1957],
        ...,
        [3269, 3067],
        [1729, 3340],
        [1429, 6448]])


In [ ]:
pos_test_list = []
with open('/content/drive/MyDrive/GSPPI_combined_train/combined_test_edge_pos_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        pos_test_list.append(tuple)

test_pos_edge = torch.tensor(pos_test_list, dtype=torch.long)
print(test_pos_edge)

3530it [00:00, 369473.04it/s]

tensor([[6442, 5619],
        [4738, 3603],
        [4738, 5164],
        ...,
        [5270,  818],
        [5270, 4743],
        [5270, 7166]])


In [ ]:
neg_test_list = []
with open('/content/drive/MyDrive/GSPPI_combined_train/combined_test_edge_neg_data.csv', 'r') as f:
    data = csv.reader(f)
    headers = next(data)
    for row in tqdm(data):
        tuple = (key_value_dict[row[0]], key_value_dict[row[1]])
        neg_test_list.append(tuple)

test_neg_edge = torch.tensor(neg_test_list, dtype=torch.long)
print(test_neg_edge)

3530it [00:00, 414893.60it/s]


tensor([[6105, 3725],
        [7284, 2424],
        [4319, 2100],
        ...,
        [4538, 3644],
        [4289, 5318],
        [ 221, 4919]])


In [ ]:
Dict = { }
Dict['train'] = {'edge':train_pos_edge , 'edge_neg': train_neg_edge}
Dict['test'] = {'edge':test_pos_edge , 'edge_neg': test_neg_edge}
print(Dict)

{'train': {'edge': tensor([[3696, 1136],
        [3696, 1409],
        [3696, 1790],
        ...,
        [1848, 2859],
        [4826, 5266],
        [7348, 5270]]), 'edge_neg': tensor([[2700,  961],
        [ 975, 2572],
        [4068, 1957],
        ...,
        [3269, 3067],
        [1729, 3340],
        [1429, 6448]])}, 'test': {'edge': tensor([[6442, 5619],
        [4738, 3603],
        [4738, 5164],
        ...,
        [5270,  818],
        [5270, 4743],
        [5270, 7166]]), 'edge_neg': tensor([[6105, 3725],
        [7284, 2424],
        [4319, 2100],
        ...,
        [4538, 3644],
        [4289, 5318],
        [ 221, 4919]])}}


In [ ]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.leaky_relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)

        return x

In [ ]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            # x = F.mish(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

pfeatures

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
feature_location = '/content/drive/MyDrive/GSPPI_combined_train/combined_graphlet_plus_pfeature_0.13.csv'
columns_to_skip = 2
feature_df = pd.read_csv(feature_location) # , usecols=range(columns_to_skip)
feature_df = feature_df.iloc[:, columns_to_skip:]
feature_df
feature_df = feature_df.dropna(axis=1)
feature_df.shape

(7576, 269)

In [ ]:
df_transformed = pd.DataFrame(feature_df)
df_transformed

,g_0,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8,g_9,...,QSO1_SC_K,QSO1_SC_L,QSO1_SC_N,QSO1_SC_P,QSO1_SC_Q,QSO1_SC_R,QSO1_SC_S,QSO1_SC_T,QSO1_SC_V,SOC1_G1
0,1,1560,0,0,3607,0,1215891,0,0,129,...,1.5792,2.2970,0.4785,2.2492,1.5314,2.1535,1.8185,1.1007,1.1964,9873.4489
1,1,1560,0,0,3607,0,1215891,0,0,129,...,1.5753,2.2913,0.4774,2.2436,1.4798,2.1958,1.8139,1.0502,1.1934,9949.1150
2,1,2014,0,0,3885,0,2026911,0,0,180,...,1.3610,4.5185,0.7077,1.6332,1.1977,1.6876,1.8509,1.1432,1.1977,11180.4989
3,10,4474,45,0,6086,35304,3330786,120,2481,322,...,0.0000,0.6831,0.6831,0.0000,0.0000,0.6831,0.6831,2.0494,1.3662,10326.5714
4,1,1560,0,0,3607,0,1215891,0,0,129,...,1.6160,2.5856,1.6160,1.1312,1.2120,1.2120,2.1008,0.7272,1.5352,10919.5282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7571,1,2,0,0,239,0,1,0,0,0,...,1.2245,4.4081,1.1755,1.2245,2.4979,1.2245,2.1061,1.1755,1.7633,10012.5180
7572,3,19,2,1,196,37,148,0,0,5,...,2.1450,1.9825,0.8775,2.0475,1.5600,1.2025,2.4700,1.9825,1.9500,9127.2404
7573,2,3574,1,0,5866,1950,3242802,0,812,309,...,1.4207,2.6692,1.8512,1.2055,0.9471,1.1193,2.2387,1.1193,1.8943,10781.2434
7574,1,1560,0,0,3607,0,1215891,0,0,129,...,1.7036,2.6601,1.0461,2.5704,1.6140,1.0760,3.0187,1.4346,2.0922,9746.9810


In [ ]:
feature_columns = df_transformed.columns
# print(feature_columns)
features_new = df_transformed[feature_columns].values
print(features_new)

[[1.00000000e+00 1.56000000e+03 0.00000000e+00 ... 1.10070000e+00
  1.19640000e+00 9.87344890e+03]
 [1.00000000e+00 1.56000000e+03 0.00000000e+00 ... 1.05020000e+00
  1.19340000e+00 9.94911500e+03]
 [1.00000000e+00 2.01400000e+03 0.00000000e+00 ... 1.14320000e+00
  1.19770000e+00 1.11804989e+04]
 ...
 [2.00000000e+00 3.57400000e+03 1.00000000e+00 ... 1.11930000e+00
  1.89430000e+00 1.07812434e+04]
 [1.00000000e+00 1.56000000e+03 0.00000000e+00 ... 1.43460000e+00
  2.09220000e+00 9.74698100e+03]
 [1.00000000e+00 5.00000000e+00 0.00000000e+00 ... 1.29940000e+00
  1.29940000e+00 1.20897736e+04]]


In [ ]:
# Convert the node features from numpy array to a PyTorch tensor
x_new = torch.tensor(features_new, dtype=torch.float) # .to_numpy()
x_new= torch.nn.functional.normalize(x_new)
min_vals, _ = torch.min(x_new, dim=0)
max_vals, _ = torch.max(x_new, dim=0)
normalized_data = (x_new - min_vals) / (max_vals - min_vals)
normalized_df = pd.DataFrame(normalized_data.numpy(), columns=df_transformed.columns)
x_pfeatures = torch.tensor(normalized_df.values, dtype=torch.float)
# print(x_new)
print(x_pfeatures.shape)

torch.Size([7576, 269])


In [ ]:
data = Data(num_nodes = G_nodes, x=x_pfeatures, adj_t = edge_index ,edge_index = edge_index )
print(data)

Data(x=[7576, 269], edge_index=[2, 17647], num_nodes=7576, adj_t=[2, 17647])


In [ ]:
hidden_channels = 256
out_channels = 128
num_layers_model = 3
num_layers_predictor = 2
dropout = 0.003
batch_size = 64 * 1024
lr = 0.002
input_dim = x_pfeatures.shape[1]
epochs = 100

In [ ]:
class LinkGNN(torch.nn.Module):
    def __init__(self, gnn, linkpred):
        super(LinkGNN, self).__init__()
        self.gnn = gnn
        self.linkpred = linkpred

    def reset_parameters(self):
        self.gnn.reset_parameters()
        self.linkpred.reset_parameters()

    def forward(self,x , edges, adj):
        h = self.gnn(x, adj)
        return self.linkpred(h[edges[0]], h[edges[1]])

In [ ]:
def build_model(data):

        gnn = SAGE(
            input_dim, hidden_channels,
            hidden_channels, num_layers_model,
            dropout)

        linkpred = LinkPredictor(
            hidden_channels, out_channels,
            1, num_layers_predictor,
            dropout)
        model = LinkGNN(gnn, linkpred)

        return model

In [ ]:
model = build_model(data)

In [ ]:
optimizer = torch.optim.Adam(list(model.parameters()) , lr=lr , weight_decay=1e-4)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
def accuracy(pos_out , neg_out):
    threshold = 0.5  # Example threshold for classification

    # Convert prediction scores to binary predictions using the threshold
    pos_predictions = (pos_out > threshold).long()
    neg_predictions = (neg_out > threshold).long()
    # print('neg_predictions--' , neg_predictions)

    # Create binary ground truth labels for positive and negative edges
    pos_labels = torch.ones_like(pos_predictions)
    neg_labels = torch.zeros_like(neg_predictions)

    # Concatenate predictions and labels for positive and negative edges
    all_predictions = torch.cat((pos_predictions, neg_predictions))
    all_labels = torch.cat((pos_labels , neg_labels))

    # Calculate accuracy
    accuracy = (all_predictions == all_labels).float().mean().item()
    # accuracy = (pos_predictions == pos_labels).float().mean().item()
    roc_auc = roc_auc_score(all_labels, all_predictions)

    return accuracy , roc_auc


In [ ]:
def metrics(pos_out , neg_out):
    threshold = 0.5  # Example threshold for classification

    # Convert prediction scores to binary predictions using the threshold
    pos_predictions = (pos_out > threshold).long()
    neg_predictions = (neg_out > threshold).long()

    # Create binary ground truth labels for positive and negative edges
    pos_labels = torch.ones_like(pos_predictions)
    neg_labels = torch.zeros_like(neg_predictions)

    # Concatenate predictions and labels for positive and negative edges
    all_predictions = torch.cat((pos_predictions, neg_predictions))
    all_labels = torch.cat((pos_labels, neg_labels))

    # Calculate true positives (TP), false positives (FP), true negatives (TN), false negatives (FN)
    TP = ((all_predictions == 1) & (all_labels == 1)).sum().item()
    FP = ((all_predictions == 1) & (all_labels == 0)).sum().item()
    TN = ((all_predictions == 0) & (all_labels == 0)).sum().item()
    FN = ((all_predictions == 0) & (all_labels == 1)).sum().item()


    # Calculate precision and recall
    precision = TP / (TP + FP) if TP + FP > 0 else 0.0
    recall = TP / (TP + FN) if TP + FN > 0 else 0.0
    if TN + FN > 0 and FP + TP > 0 and TN + FP > 0 and FN + TP > 0:
        mcc_met = ((TP * TN) - (FP * FN)) / math.sqrt((TN + FN) * (FP + TP) * (TN + FP) * (FN + TP))
    else:
        mcc_met = 0.0

    confusion_matrix = np.array([[TP, FP], [FN, TN]])
    class_names = ['Positive', 'Negative']
    # plot_confusion_matrix(confusion_matrix, class_names)

    return precision , recall , mcc_met

In [ ]:
def plot_confusion_matrix(confusion_matrix, class_names):
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names)
    plt.yticks(tick_marks, class_names)

    thresh = confusion_matrix.max() / 2.0
    # for i, j in np.ndenumerate(confusion_matrix):
    #     plt.text(j, i, f'{confusion_matrix[i, j]}', horizontalalignment='center', color='white' if confusion_matrix[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()



In [ ]:
from torch_geometric.utils import negative_sampling, to_undirected

In [ ]:
def train(model, data, split_edge, optimizer, batch_size , epoch):
    model.train()
    # predictor.train()

    total_loss = total_examples = 0
    pos_train_preds = []
    neg_train_preds = []
    pos_train_edge = split_edge['train']['edge']#.to(data.x)#.device)
    neg_train_edge = split_edge['train']['edge_neg']#.to(data.x)#.device)

    # scheduler = MultiStepLR(optimizer, milestones=[1,5], gamma=0.5)
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size, shuffle=True):
        #print('perm--', perm)
        optimizer.zero_grad()
        pos_edge = pos_train_edge[perm].t()
        neg_edge = neg_train_edge[perm].t()
        # neg_edge = negative_sampling(edge_index, num_nodes=data.num_nodes, num_neg_samples=pos_edge.size(1), method='dense')
        out = model( data.x, torch.cat([pos_edge, neg_edge], 1), data.adj_t).squeeze()

        pos_out = out[:pos_edge.size(1)]
        pos_loss = -torch.log(pos_out + 1e-8).mean()

        neg_out = out[pos_edge.size(1):]
        neg_loss = -torch.log(1 - neg_out + 1e-8).mean()

        loss = pos_loss + neg_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        #accuracy
        train_accuracy , train_rocauc = accuracy(pos_out , neg_out)
        train_precision , train_recall , train_mcc = metrics(pos_out , neg_out)
        f1_score = 2 * (train_precision * train_recall) / (train_precision + train_recall) if (train_precision + train_recall) > 0 else 0.0

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples
        avg_loss_train = total_loss / total_examples

        print(f'Epoch: {epoch:02d}, ', f'Train loss: {avg_loss_train:.4f}, ',  f'Train accuracy: {train_accuracy:.4f}, '
              f'Train precision: {train_precision:.4f}, ' , f'Train recall: {train_recall:.4f}, ' , f'Train mcc: {train_mcc:.4f}, '
              , f'Train f1_score: {f1_score:.4f}, ' ,f'Train rocauc: {train_rocauc:.4f}, ' )
    # scheduler.step()

    return train_accuracy , total_loss / total_examples

In [ ]:
@torch.no_grad()
def test(model, data, split_edge, evaluator, batch_size):
    model.eval()

    pos_test_edge = split_edge['test']['edge']#.to(data.x)#.device)
    neg_test_edge = split_edge['test']['edge_neg']#.to(data.x)#.device)

    total_loss = total_examples = 0
    test_accuracy = test_precision = test_recall = 0.0
    #pos_test_preds , neg_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size, shuffle=False):

        pos_edge = pos_test_edge[perm].t()
        neg_edge = neg_test_edge[perm].t()
        # neg_edge = negative_sampling(edge_index, num_nodes=data.num_nodes, num_neg_samples=pos_edge.size(1), method='dense')
        out = model( data.x, torch.cat([pos_edge, neg_edge], 1), data.adj_t).squeeze()

        pos_out = out[:pos_edge.size(1)]
        pos_loss = -torch.log(pos_out + 1e-8).mean()

        neg_out = out[pos_edge.size(1):]
        neg_loss = -torch.log(1 - neg_out + 1e-8).mean()

        loss = pos_loss + neg_loss
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #accuracy
        test_accuracy ,test_rocauc = accuracy(pos_out , neg_out)
        test_precision , test_recall , test_mcc = metrics(pos_out , neg_out)
        f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall) if (test_precision + test_recall) > 0 else 0.0

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

        print(f'Test accuracy: {test_accuracy:.4f}, '
              f'Test precision: {test_precision:.4f}, ' , f'Test recall: {test_recall:.4f}, ' , f'Test mcc: {test_mcc:.4f}, ' ,
              f'Test f1_score: {f1_score:.4f}, ' , f'Test rocauc: {test_rocauc:.4f}, ' )

    return test_accuracy , total_loss / total_examples

In [ ]:
data

Data(x=[7576, 269], edge_index=[2, 17647], num_nodes=7576, adj_t=[2, 17647])

In [ ]:
class ModelCheckpoint:
    def __init__(self, model_save_load_dir ):
        self.model_dir = model_save_load_dir
        self.num_epochs = 0
        self.metric_arr = []
        self.saved_at = ""
        self.model_state_dict_key = 'model_state_dict'
        self.optimizer_state_dict_key = 'optimizer_state_dict'
        self.metric_dict_key = 'metric'
        self.num_epochs_key = 'num_epochs'
        self.metric = -np.inf
        self.model_state = None
        self.optim_state = None


    def get_metric_arr(self):
        return self.metric_arr

    def get_last_saved_loc(self):
        return self.saved_at

    def get_num_epochs(self):
        return self.num_epochs

    def load_checkpoint(self, model_load_file_name : str = "some.pth.tar", model = None, optim = None):
        self.saved_at = model_load_file_name
        checkpoint = torch.load(self.saved_at)
        self.num_epochs = checkpoint[self.num_epochs_key]
        self.metric = checkpoint[self.metric_dict_key]
        model.load_state_dict(checkpoint[self.model_state_dict_key])
        self.model_state = copy.deepcopy(model)
        if (optim != None):
            optim.load_state_dict(checkpoint[self.optimizer_state_dict_key])
            self.optim_state = copy.deepcopy(optim)

        print(f"Model loaded successfully from checkpoint with metric = {self.metric:.5f}, num_epochs = {self.num_epochs:03d}...")

    def save_checkpoint(self):
        print("=> Saving a new best")
        # save_file_name = f'{self.model_save_prefix}_epoch_{self.num_epochs:03d}_metric_{self.metric:.5f}.pth.tar'
        save_at = self.model_dir
        if (self.optim_state != None):
            state_dict = {self.num_epochs_key : self.num_epochs,
                        self.model_state_dict_key : self.model_state.state_dict(),
                        self.optimizer_state_dict_key : self.optim_state.state_dict(),
                        self.metric_dict_key : self.metric}
        else:
            state_dict = {self.num_epochs_key : self.num_epochs,
                        self.model_state_dict_key : self.model_state.state_dict(),
                        self.metric_dict_key : self.metric}
        torch.save(state_dict, save_at)
        self.saved_at = save_at
        print("Best model saved at location : ", save_at)


    def change_current(self, is_best, metric, state, optim):
        if is_best:
            self.metric = metric
            self.model_state = copy.deepcopy(state)
            if optim != None :
                self.optim_state = copy.deepcopy(optim)
            self.save_checkpoint()


    def create_checkpoint(self, state, metric, epoch, optim = None):
        self.num_epochs = epoch
        self.metric_arr.append(self.metric)
        self.change_current(True, metric, state, optim)


In [ ]:
min_loss_train = 100
best_accuracy_train = 0.0
min_loss_epoch_train = 80
best_acc_epoch_train = 0

min_loss_valid = 100
best_accuracy_valid = 0.0
min_loss_epoch_valid = 80
best_acc_epoch_valid = 0


#early stopping
n_epochs_stop = 20
epochs_no_improve = 0
early_stop = False
model_save_path = '/content/drive/MyDrive/GSPPI_combined_train/combined_train_ensemble_0.13.pth.tar'
modelcheckpoint = ModelCheckpoint(model_save_load_dir=model_save_path)
for epoch in range(1, 1 + epochs):
    train_accuracy , train_loss = train(model, data, Dict, optimizer, batch_size , epoch)

    if(train_accuracy > best_accuracy_train):
      # print("Inside accuracy > best_accuracy")
      best_accuracy_train = train_accuracy
      best_acc_epoch_train = epoch
      # path to save the model
      modelcheckpoint.create_checkpoint(model, best_accuracy_train, epoch, optimizer)

    if(train_loss < min_loss_train):
      # print("loss< min_loss")
      epochs_no_improve = 0
      min_loss_train = train_loss
      min_loss_epoch_train = epoch
    elif train_loss >= min_loss_train :
      print("loss> min_loss")
      epochs_no_improve += 1

    if epoch > 5 and epochs_no_improve == n_epochs_stop:
      print('Early stopping!' )
      early_stop = True
      break

modelcheckpoint.create_checkpoint(model, best_accuracy_train, epoch, optimizer)

print(f'min_train_loss : {min_loss_train} for epoch {min_loss_epoch_train} ............... best_train_accuracy : {best_accuracy_train} for epoch {best_acc_epoch_train}')
print("Model saved")

Epoch: 01,  Train loss: 1.3871,  Train accuracy: 0.5000, Train precision: 0.5000,  Train recall: 1.0000,  Train mcc: 0.0000,  Train f1_score: 0.6667,  Train rocauc: 0.5000, 
=> Saving a new best
Best model saved at location :  /content/drive/MyDrive/GSPPI_combined_train/combined_train_ensemble_0.13.pth.tar
Epoch: 02,  Train loss: 1.3848,  Train accuracy: 0.5000, Train precision: 0.5000,  Train recall: 1.0000,  Train mcc: 0.0000,  Train f1_score: 0.6667,  Train rocauc: 0.5000, 
Epoch: 03,  Train loss: 1.3785,  Train accuracy: 0.5081, Train precision: 0.5041,  Train recall: 0.9890,  Train mcc: 0.0591,  Train f1_score: 0.6678,  Train rocauc: 0.5081, 
=> Saving a new best
Best model saved at location :  /content/drive/MyDrive/GSPPI_combined_train/combined_train_ensemble_0.13.pth.tar
Epoch: 04,  Train loss: 1.3585,  Train accuracy: 0.7047, Train precision: 0.6467,  Train recall: 0.9025,  Train mcc: 0.4458,  Train f1_score: 0.7535,  Train rocauc: 0.7047, 
=> Saving a new best
Best model save

In [ ]:
@torch.no_grad()
def test_new_model(model, data, split_edge, optimizer, batch_size):
    model.eval()

    pos_test_edge = split_edge['test']['edge']
    neg_test_edge = split_edge['test']['edge_neg']

    total_loss = total_examples = 0
    test_accuracy = test_precision = test_recall = 0.0
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size, shuffle=False):

        pos_edge = pos_test_edge[perm].t()
        # neg_edge = negative_sampling(edge_index, num_nodes=data.num_nodes, num_neg_samples=pos_edge.size(1), method='dense')
        neg_edge = neg_test_edge[perm].t()
        out = model( data.x, torch.cat([pos_edge, neg_edge], 1), data.adj_t).squeeze()

        pos_out = out[:pos_edge.size(1)]
        pos_loss = -torch.log(pos_out + 1e-8).mean()

        neg_out = out[pos_edge.size(1):]
        neg_loss = -torch.log(1 - neg_out + 1e-8).mean()

        loss = pos_loss + neg_loss
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        #accuracy
        test_accuracy , test_rocauc = accuracy(pos_out , neg_out)
        test_precision, test_recall, test_mcc = metrics(pos_out , neg_out)

        if(test_precision + test_recall) > 0:
           f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
        else:
           f1_score = 0.0

        num_examples = pos_out.size(0)
        total_loss = total_loss+loss.item() * num_examples
        total_examples= total_examples+num_examples
        avg_loss_test = total_loss / total_examples

        print('Evaluation on saved model')
        print( f'Test loss: {avg_loss_test:.4f}, ',  f'Test accuracy: {test_accuracy:.4f}, ' , f'Test precision: {test_precision:.4f}, ' , f'Test recall: {test_recall:.4f}, ' , f'Test mcc: {test_mcc:.4f}, ' , f'Test f1_score: {f1_score:.4f}, ' , f'Test rocauc_score: {test_rocauc:.4f}, '  )

    return total_loss / total_examples

In [ ]:
model_save_path = '/content/drive/MyDrive/GSPPI_combined_train/combined_train_ensemble_0.13.pth.tar'
modelcheckpoint = ModelCheckpoint(model_save_path)
model = build_model(data)
modelcheckpoint.load_checkpoint(model_save_path , model, optimizer)
print(model)

Model loaded successfully from checkpoint with metric = 0.93295, num_epochs = 100...
LinkGNN(
  (gnn): SAGE(
    (convs): ModuleList(
      (0): SAGEConv(269, 256, aggr=mean)
      (1-2): 2 x SAGEConv(256, 256, aggr=mean)
    )
  )
  (linkpred): LinkPredictor(
    (lins): ModuleList(
      (0): Linear(in_features=256, out_features=128, bias=True)
      (1): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)


In [ ]:
loss = test_new_model(model, data, Dict, optimizer, batch_size)

Evaluation on saved model
Test loss: 0.6557,  Test accuracy: 0.9125,  Test precision: 0.8848,  Test recall: 0.9484,  Test mcc: 0.8271,  Test f1_score: 0.9155,  Test rocauc_score: 0.9125, 
